In [1]:
import math
import re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from joblib import Parallel, delayed
from tqdm import tqdm
from nltk.corpus import stopwords  # Import the stop word list
import re

from sklearn.metrics import accuracy_score 
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import log_loss
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import make_scorer

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import make_pipeline
from sklearn.base import TransformerMixin

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import ExtraTreesClassifier, ExtraTreesRegressor
from sklearn.ensemble import AdaBoostClassifier, AdaBoostRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from keras.wrappers.scikit_learn import KerasClassifier
from keras.wrappers.scikit_learn import KerasRegressor
from keras.layers import Flatten, Dense, Activation, Dropout
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras.callbacks import Callback
from keras.layers.advanced_activations import LeakyReLU
from keras.callbacks import TensorBoard  
from keras import backend as K
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier, XGBRegressor

import lightgbm as lgb
import catboost as cb
import xgboost as xgb

%matplotlib inline
plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams["figure.figsize"] = (12, 9)
sns.set(context='paper', style='darkgrid', rc={'figure.facecolor':'white'}, font_scale=1.2)

# import logging       
# logging.basicConfig(filename='./my_framework.txt', filemode='a', datefmt='%H:%M:%S', level=logging.DEBUG,
#                     format='%(asctime)s %(levelname)s %(message)s')

/home/mrqtros/Software/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/mrqtros/Software/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.4 of module '_catboost' does not match runtime version 3.6
  return f(*args, **kwds)


# Dataset preparation

In [2]:
dftrain = pd.read_csv("input/train.csv")
dftrain['comment_text'] = dftrain['comment_text'].apply(str) # some values parsed as float
dftest = pd.read_csv("input/test.csv")
dftest['comment_text'] = dftest['comment_text'].apply(str) # some values parsed as float

In [4]:
english_stops = set(stopwords.words("english"))
def preprocess_line(raw_line):
    raw_line = re.sub('\\b\\d+\\b', ' ', raw_line)
    raw_line = re.sub('[\';:.,<>#*"\-=/?!№\[\]()«»_|\\\\…•+%]', ' ', raw_line)
    words = raw_line.lower().split()
    meaningful_words = [w for w in words if w not in english_stops]
    return " ".join(meaningful_words)

In [5]:
# TODO IS IT WORTH DOING?
dftrain['comment_text_preprocessed'] = dftrain['comment_text'].apply(preprocess_line)
dftest['comment_text_preprocessed'] = dftest['comment_text'].apply(preprocess_line)

In [6]:
dftrain.head()
# dftrain['comment_text_preprocessed'][0], dftrain['comment_text'][0]

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,comment_text_preprocessed
0,22256635,"Nonsense? kiss off, geek. what I said is true...",1,0,0,0,0,0,nonsense kiss geek said true account terminated
1,27450690,"""\n\n Please do not vandalize pages, as you di...",0,0,0,0,0,0,please vandalize pages edit w merwin continue ...
2,54037174,"""\n\n """"Points of interest"""" \n\nI removed the...",0,0,0,0,0,0,points interest removed points interest sectio...
3,77493077,Asking some his nationality is a Racial offenc...,0,0,0,0,0,0,asking nationality racial offence wow aware bl...
4,79357270,The reader here is not going by my say so for ...,0,0,0,0,0,0,reader going say ethereal vocal style dark lyr...


# Dummy pass

In [17]:
vectorizer = TfidfVectorizer(max_features=60000)
X = vectorizer.fit_transform(dftrain['comment_text_preprocessed'])
y = dftrain['toxic']

In [18]:
clf = MultinomialNB()
clf.fit(X, y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [21]:
prediction = clf.predict_proba(X)

In [22]:
log_loss(y, prediction)

0.18122484797361807

# Rare word Analysis

In [23]:
word_counts = {}
for sentence in dftrain['comment_text']:
    sentence = sentence.lower()
    for word in sentence.split():
        word = word.strip('0123456789,." /:;-•()|—=&%!?~\'+·{}#[]<>*@←→\\☎^_£✉►…♣©♦†☼¢♠♥™à✍«»₪±∇∆♫°≠ω`´≈✎★ᛏ☯✔')
        if word in word_counts:
            word_counts[word] += 1
        else:
            word_counts[word] = 1
plain_list = sorted(word_counts.items(), key=lambda x: x[1])

In [24]:
with open('plain_my.txt', 'w') as plain_out:
    for w,c in plain_list:
        plain_out.write('%6s %s\n' % (c,w))

In [ ]:

118 ===d~~penis
60 fc*k 
54 faggot!!!!jéské (https://encyclopediadramatica.rs/J%C3%A9sk%C3%A9_Couriano)
50 font-size:85% (53 style=""background style=""background-color 43 style=""border-spacing:8px;margin:0px 29 style=""vertical-align)
46 lunchables!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!1lol
41 t@lk 13 †@1₭ 14 трёп
38 ☺ 18 ｡◕‿◕｡
37 asshole!!!this
34 freedom!freedom!freedom!freedom!freedom!freedom!freedom!freedom!freedom!freedom!freedom!freedom!freedom!freedom!freedom!freedom!
33 f@ggot
32 zzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzz
30 f*** 30 f***ing 9 bullsh*t 18 fu*k 36 f*cking 60 fc*k 69 f*ck 1 f*ucked 1 su**s 1 s*#t 1 nig*ger 1 mothaf*cka 
27 fuck!{{unblock}}lifetime
24 you!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
22 f**k 12 sh*t 15 f**king 22 *** 8 a**hole 11 fuck's 
19 fuckfuckfuckfuckfuckfuckfuckfuckfuckfuckfuckfuckfuckfuckfuckfuckfuckfuckfuckfuckfuckfuckfuckfuck
18 #fdffe 12 #eef 12 #fee 12 #dde 12 #edd
14 http://rexcurry.net/pledge2.html
12 bullshido
10 gayreek (https://www.urbandictionary.com/define.php?term=gayreek)
10 kool
10 suki (Даааааааааааааа :) )

In [ ]:
https://justenglish.me/2014/07/18/lol-omg-and-ily-60-of-the-dominating-abbreviations/ 
wtf
omfg
ffs
stfu
wth
ftw
mf (motherfucker)
gtfo


In [ ]:
# Отдельно все запиканные
=      1 a***es
=      1 a**hoe
=      1 a*fgan
=      1 a*s
=      1 apparently*cough
=      1 ars*hole
=      1 as**ole
=      1 as*fu*king
=      1 as*hol*s
=      1 as*hole
=      1 ass*hole
=      1 assh*le's
=      1 assh*ole
=      1 b******d
=      1 b***er
=      1 b**ching
=      1 b**tard
=      1 b*ch
=      1 b*lls
=      1 b*stard
=      1 b*ugger
=      1 ba*tard
=      1 beyond*the
=      1 bi*ch
=      1 bl**dy
=      1 bulls*it
=      1 c*-algebras
=      1 c*ck
=      1 c*itadel
=      1 c*ntfaced
=      1 c*nts
=      1 clusterf*ck
=      1 cocksu*_cking
=      1 cough*...(yes
=      1 cough*encylopedia
=      1 cough*like
=      1 cr*p
=      1 d***s
=      1 d**b**s
=      1 d*ck
=      1 d*irty
=      1 d*mb
=      1 d*mn
=      1 di*ks
=      1 dqa1*0501:dqb
=      1 dqa1*0505:dqb
=      1 edio*magica
=      1 edito*magica
=      1 f*(&n
=      1 f*(x
=      1 f**%!ng
=      1 f*****g
=      1 f****er
=      1 f****r
=      1 f***ers
=      1 f***in
=      1 f***king
=      1 f**ck
=      1 f**k's
=      1 f**k**g
=      1 f**kin
=      1 f**kker
=      1 f*ck*ng
=      1 f*ck.and
=      1 f*ckhead
=      1 f*ckin
=      1 f*ggots
=      1 f*u*c*k
=      1 f*ucked
=      1 f*ucking
=      1 f_uc*ck
=      1 facts*”
=      1 fagg*t
=      1 fu*er
=      1 fu*khead
=      1 fu*king
=      1 fu*kwit
=      1 fuckfu*ker
=      1 g***y
=      1 g*d
=      1 g*y
=      1 gam*boy
=      1 h*ll
=      1 h*r
=      1 h*tler
=      1 haven*t
=      1 hon*ey
=      1 id*ot
=      1 id*t
=      1 j*sus
=      1 khasnya*thud
=      1 l*a*b
=      1 m*'s
=      1 m*(1+i
=      1 m*(1+r)*(1+p
=      1 m*n
=      1 ma**yks
=      1 macy*s
=      1 moderf***n
=      1 mot*herfuc*ker
=      1 moth*rfucker
=      1 mothaf*cka
=      1 mother****er
=      1 motherf***in
=      1 motherf**ker
=      1 motherf**king
=      1 motherf*cking
=      1 motherfu*_ccker
=      1 musashi*]'s
=      1 n*****r
=      1 n****r
=      1 n**ger
=      1 n*gga
=      1 n*m
=      1 n*zi
=      1 nig*er
=      1 nig*ger
=      1 nudge*”
=      1 p**r**b***y
=      1 p**tang
=      1 p*erverts
=      1 p*opd*ck
=      1 p*rverts
=      1 p*ss
=      1 page*-they
=      1 per*ert
=      1 pi*g
=      1 pi*n
=      1 pr***s
=      1 pri*k
=      1 q1*q2/(4pir
=      1 r*(meters/ohm
=      1 r*t*rd
=      1 re***d
=      1 ret*rds
=      1 s*#t
=      1 s*****”
=      1 s***-standard
=      1 s*cks
=      1 s*d
=      1 s*it
=      1 septegram*talk*contributions
=      1 sh**brained
=      1 sh**t
=      1 sh*thole
=      1 sh*ting
=      1 sh*tty
=      1 starfu*k
=      1 su**s
=      1 swifty*talk
=      1 t*rd
=      1 t*ts
=      1 t_amb*(gamma
=      1 tr***s
=      1 u*v
=      1 un*sexyness
=      1 v*rgins
=      1 wh*r
=      1 wh*re
=      1 yaj*mart
=      2 a***-retentive
=      2 b*ll
=      2 c*******ing
=      2 c*****shhh
=      2 c***s
=      2 c**t
=      2 co**sucker
=      2 cr*ppy
=      2 doesn*t
=      2 f***er
=      2 f**in
=      2 f*k
=      2 f*p
=      2 ri*(meters/ohm
=      2 s*n
=      2 t*m
=      2 t*n
=      2 tw*t
=      3 assh*le
=      3 b***h
=      3 bulls**t
=      3 c*nt
=      3 d**n
=      3 di*k
=      3 f**cking
=      3 fuc*in
=      3 motherf*cker
=      3 t*u
=      3 wal*mart
=      4 b**ch
=      4 f*cked
=      4 f*cker
=      4 f*ing
=      4 fu*ker.sikh
=      4 fuc*ers
=      4 y*z
=      5 b*tch
=      5 fuc*ing
=      6 f**ks
=      6 fu*ker
=      6 s**t
=      7 m*a*s*h
=      8 a**hole
=      9 bullsh*t
=     15 f**king
=     17 sh*t
=     18 fu*k
=     22 f**k
=     30 f***ing
=     36 f*cking
=     60 fc*k
=     69 f*ck

# Grid search

In [7]:
X = dftrain['comment_text_preprocessed']
y = y = dftrain['toxic']

In [8]:
def gscv_summary(grid, print_all=False, name='UKNWN'):
    print("[%s] Best score/params: %s %s" % (name, grid.best_score_, grid.best_params_))
#     logging.debug("[%s] Best score/params: %s %s" % (name, grid.best_score_, grid.best_params_))
    if not print_all: 
        return
    print("All params:")
    means = grid.cv_results_['mean_test_score']
    stds = grid.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, grid.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))

In [9]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('mnb', MultinomialNB())
])

In [18]:
%%time

param_grid = {
    'tfidf__max_features': [6400, 6500, 6600],
#     'tfidf__stop_words': ['english', None],
    'tfidf__binary': [True, False],
    'tfidf__norm': ['l2'],
    'tfidf__smooth_idf': [True, False],
    'tfidf__sublinear_tf': [True],
    'tfidf__use_idf': [True],
#     'mnb__alpha': [0.0, 1.0, 2.0],
#     'mnb__fit_prior': [True, False],
}
grid = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=5, scoring='neg_log_loss', n_jobs=-1, verbose=2)
grid.fit(X, y)
gscv_summary(grid, name='MNB')

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] tfidf__binary=True, tfidf__max_features=6400, tfidf__norm=l2, tfidf__smooth_idf=True, tfidf__sublinear_tf=True, tfidf__use_idf=True 
[CV] tfidf__binary=True, tfidf__max_features=6400, tfidf__norm=l2, tfidf__smooth_idf=True, tfidf__sublinear_tf=True, tfidf__use_idf=True 
[CV] tfidf__binary=True, tfidf__max_features=6400, tfidf__norm=l2, tfidf__smooth_idf=True, tfidf__sublinear_tf=True, tfidf__use_idf=True 
[CV] tfidf__binary=True, tfidf__max_features=6400, tfidf__norm=l2, tfidf__smooth_idf=True, tfidf__sublinear_tf=True, tfidf__use_idf=True 
[CV] tfidf__binary=True, tfidf__max_features=6400, tfidf__norm=l2, tfidf__smooth_idf=True, tfidf__sublinear_tf=True, tfidf__use_idf=True 
[CV] tfidf__binary=True, tfidf__max_features=6400, tfidf__norm=l2, tfidf__smooth_idf=False, tfidf__sublinear_tf=True, tfidf__use_idf=True 
[CV] tfidf__binary=True, tfidf__max_features=6400, tfidf__norm=l2, tfidf__smooth_idf=False, tfidf__sublinear_t

[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   10.2s


[CV] tfidf__binary=True, tfidf__max_features=6600, tfidf__norm=l2, tfidf__smooth_idf=False, tfidf__sublinear_tf=True, tfidf__use_idf=True 
[CV] tfidf__binary=True, tfidf__max_features=6600, tfidf__norm=l2, tfidf__smooth_idf=False, tfidf__sublinear_tf=True, tfidf__use_idf=True 
[CV] tfidf__binary=True, tfidf__max_features=6600, tfidf__norm=l2, tfidf__smooth_idf=False, tfidf__sublinear_tf=True, tfidf__use_idf=True 
[CV]  tfidf__binary=True, tfidf__max_features=6500, tfidf__norm=l2, tfidf__smooth_idf=True, tfidf__sublinear_tf=True, tfidf__use_idf=True, total=   5.5s
[CV] tfidf__binary=True, tfidf__max_features=6600, tfidf__norm=l2, tfidf__smooth_idf=False, tfidf__sublinear_tf=True, tfidf__use_idf=True 
[CV]  tfidf__binary=True, tfidf__max_features=6500, tfidf__norm=l2, tfidf__smooth_idf=True, tfidf__sublinear_tf=True, tfidf__use_idf=True, total=   6.1s
[CV] tfidf__binary=True, tfidf__max_features=6600, tfidf__norm=l2, tfidf__smooth_idf=False, tfidf__sublinear_tf=True, tfidf__use_idf=True 

[CV] tfidf__binary=False, tfidf__max_features=6600, tfidf__norm=l2, tfidf__smooth_idf=True, tfidf__sublinear_tf=True, tfidf__use_idf=True 
[CV]  tfidf__binary=False, tfidf__max_features=6500, tfidf__norm=l2, tfidf__smooth_idf=True, tfidf__sublinear_tf=True, tfidf__use_idf=True, total=   5.4s
[CV] tfidf__binary=False, tfidf__max_features=6600, tfidf__norm=l2, tfidf__smooth_idf=False, tfidf__sublinear_tf=True, tfidf__use_idf=True 
[CV]  tfidf__binary=False, tfidf__max_features=6400, tfidf__norm=l2, tfidf__smooth_idf=False, tfidf__sublinear_tf=True, tfidf__use_idf=True, total=   6.2s
[CV]  tfidf__binary=False, tfidf__max_features=6400, tfidf__norm=l2, tfidf__smooth_idf=False, tfidf__sublinear_tf=True, tfidf__use_idf=True, total=   6.2s
[CV] tfidf__binary=False, tfidf__max_features=6600, tfidf__norm=l2, tfidf__smooth_idf=False, tfidf__sublinear_tf=True, tfidf__use_idf=True 
[CV] tfidf__binary=False, tfidf__max_features=6600, tfidf__norm=l2, tfidf__smooth_idf=False, tfidf__sublinear_tf=True

[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:   37.5s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:   37.5s finished


[MNB] Best score/params: -0.14088461628285465 {'tfidf__binary': False, 'tfidf__max_features': 6500, 'tfidf__norm': 'l2', 'tfidf__smooth_idf': False, 'tfidf__sublinear_tf': True, 'tfidf__use_idf': True}
CPU times: user 6.18 s, sys: 1.39 s, total: 7.57 s
Wall time: 41.2 s


# Kernel NBSVM

https://www.kaggle.com/jhoward/nb-svm-strong-linear-baseline-eda-0-052-lb

In [19]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_is_fitted
from sklearn.linear_model import LogisticRegression
from scipy import sparse
class NbSvmClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, C=1.0, dual=False, n_jobs=1):
        self.C = C
        self.dual = dual
        self.n_jobs = n_jobs

    def predict(self, x):
        # Verify that model has been fit
        check_is_fitted(self, ['_r', '_clf'])
        return self._clf.predict(x.multiply(self._r))

    def predict_proba(self, x):
        # Verify that model has been fit
        check_is_fitted(self, ['_r', '_clf'])
        return self._clf.predict_proba(x.multiply(self._r))

    def fit(self, x, y):
        # Check that X and y have correct shape
        y = y.values
        x, y = check_X_y(x, y, accept_sparse=True)

        def pr(x, y_i, y):
            p = x[y==y_i].sum(0)
            return (p+1) / ((y==y_i).sum()+1)

        self._r = sparse.csr_matrix(np.log(pr(x,1,y) / pr(x,0,y)))
        x_nb = x.multiply(self._r)
        self._clf = LogisticRegression(C=self.C, dual=self.dual, n_jobs=self.n_jobs).fit(x_nb, y)
        return self

In [20]:
import pandas as pd, numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [22]:
train = pd.read_csv('./input/train.csv')
test = pd.read_csv('./input/test.csv')
subm = pd.read_csv('./input/sample_submission.csv')

In [23]:
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
train['none'] = 1-train[label_cols].max(axis=1)
train.describe()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate,none
count,9.585100e+04,95851.000000,95851.000000,95851.000000,95851.000000,95851.000000,95851.000000,95851.000000
mean,4.994359e+11,0.096368,0.010068,0.053301,0.003182,0.049713,0.008492,0.897862
std,2.890136e+11,0.295097,0.099832,0.224635,0.056320,0.217352,0.091762,0.302831
min,2.225664e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.473437e+11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,5.001297e+11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,7.501088e+11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,9.999882e+11,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [24]:
COMMENT = 'comment_text'
train[COMMENT].fillna("unknown", inplace=True)
test[COMMENT].fillna("unknown", inplace=True)

In [26]:
import re, string
re_tok = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
def tokenize(s): 
    return re_tok.sub(r' \1 ', s).split()

In [27]:
n = train.shape[0]
vec = TfidfVectorizer(ngram_range=(1,2), tokenizer=tokenize,
               min_df=3, max_df=0.9, strip_accents='unicode', use_idf=True,
               smooth_idf=True, sublinear_tf=True)
trn_term_doc = vec.fit_transform(train[COMMENT])
test_term_doc = vec.transform(test[COMMENT])

In [28]:
len(vec.vocabulary_)

285100

In [38]:
# preds = np.zeros((len(test), len(label_cols)))
cvs = []
for i, column in enumerate(label_cols):
    print('Fit:', column)
    model = NbSvmClassifier()
    model.fit(trn_term_doc, train[column])
    pred = model.predict_proba(trn_term_doc)
    ll = log_loss(train[column], pred)
    print(column, "log loss:", ll)
    scores = cross_val_score(model, trn_term_doc, train[column], cv=5, scoring='neg_log_loss')
    print(f"CV mean {scores.mean()} std {scores.std()}\nraw {scores}")
    cvs.append(scores.mean())
#     preds[:,i] = m.predict_proba(test_x.multiply(r))[:,1]

Fit: toxic
toxic log loss: 0.0683931911152053
CV mean -0.10984694199293106 std 0.002453268100769032
raw [-0.11233353 -0.11285825 -0.10970464 -0.10765906 -0.10667923]
Fit: severe_toxic
severe_toxic log loss: 0.008496710919669564
CV mean -0.029764572797230233 std 0.0017811032304001543
raw [-0.03100353 -0.03099577 -0.03155157 -0.02816316 -0.02710884]
Fit: obscene
obscene log loss: 0.031324106570019404
CV mean -0.06168139148446224 std 0.0026725920672013596
raw [-0.06027316 -0.05797601 -0.0643994  -0.06508642 -0.06067196]
Fit: threat
threat log loss: 0.0020182706494972194
CV mean -0.011488439300173603 std 0.0008807495410935281
raw [-0.01236054 -0.01243243 -0.01001941 -0.0113807  -0.01124912]
Fit: insult
insult log loss: 0.038527573970870324
CV mean -0.07788704113017045 std 0.0015969837820944425
raw [-0.07649849 -0.07868979 -0.07622583 -0.08057502 -0.07744607]
Fit: identity_hate
identity_hate log loss: 0.007024088231383722
CV mean -0.028086126746360927 std 0.0005808056435436677
raw [-0.02803

In [39]:
print(sum(cvs) / len(cvs))

-0.053125752241888084
